<a href="https://colab.research.google.com/github/nigoda/machine_learning/blob/main/12_NLP_Load_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load text**

Next, you will use lower-level utilities like *tf.data.TextLineDataset* to load text files, and *tf.text* to preprocess the data for finer-grain control

In [ ]:
import collections
import pathlib
import re
import string

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds

In [ ]:
!pip install tensorflow-text

     |████████████████████████████████| 3.4MB 5.0MB/s 


In [ ]:
import tensorflow_text as tf_text

## Example 1: Predict the tag for a stack Overflow question

### Download and explore the dataset

In [ ]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset = utils.get_file(
    'stack_overflow_16k.tar.gz',
    data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir=''
)
dataset_dir = pathlib.Path(dataset).parent

6053888/6053168 [==============================] - 0s 0us/step


In [ ]:
list(dataset_dir.iterdir())

[PosixPath('/tmp/.keras/test'),
 PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz.tar.gz'),
 PosixPath('/tmp/.keras/train'),
 PosixPath('/tmp/.keras/README.md')]

In [ ]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[PosixPath('/tmp/.keras/train/csharp'),
 PosixPath('/tmp/.keras/train/javascript'),
 PosixPath('/tmp/.keras/train/java'),
 PosixPath('/tmp/.keras/train/python')]

The train/cshape, train/java, train/python and train/javascript directories contain many text files, each of which is a stack Overflow question. Print a file and inspect the data.

In [ ]:
sample_file = train_dir/'python/1775.txt'
with open(sample_file) as f:
  print(f.read())

"check if a numbers comes before the same number but am not getting any output from the code number =[2,3,4,9,9,5,1].def checklist(list1):.    for i in range(len(list1 - 1)):..if list1[i] == 9 and list1[i+1] == 9:.            return true.    return false...this code does not output any value either true or false, it is suppose to output true if 9 comes after 9 or false if otherwise .[enter image description here][1]..[1]: https://i.stack.imgur.com/kwm0s.png  this link contain the code and the output"



### Load the dataset

Next, you will load the data off disk and prepare it into a format suitable for training. To do so, you will use *text_dataset_from_directory* utility to create a labeled *tf.data.Dataset*. If you're new to tf.data, it's a power collection of tools for building input pipeline.

The *preprocessing.text_dataset_from_directory* expect a directory structure as follows.

train/

...csharp/
......1.txt
......2.txt

...java/
......1.txt
......2.txt

...javascript/
......1.txt
......2.txt

...python/
......1.txt
......2.txt

When running a ML experiment,it is a best practice to divide your dataset into three splits:train,validation, and test. The Stack Overflow dataset has already been divided into train and test,but it lacks a validation set. Create a validation set using an 80:20 split of the training data by using the validation_split argument below.



In [ ]:
batch_size = 32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size = batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
) 

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


There are 8,000 examples in the training folder, of which you will use 80%(or 6,400) for training. 

Train a model by passing a tf.data.Dataset directly to model.fit.

First, iterate over the dataset and print out a few examples, to get a feel for the data. 

In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    print("Question : ",text_batch.numpy()[i][:100], '...')
    print("Labels : ",label_batch.numpy()[i])

Question :  b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can' ...
Labels :  1
Question :  b'"blank code slow skin detection this code changes the color space to lab and using a threshold finds' ...
Labels :  3
Question :  b'"option and validation in blank i want to add a new option on my system where i want to add two text' ...
Labels :  1
Question :  b'"exception: dynamic sql generation for the updatecommand is not supported against a selectcommand th' ...
Labels :  0
Question :  b'"parameter with question mark and super in blank, i\'ve come across a method that is formatted like t' ...
Labels :  1
Question :  b'call two objects wsdl the first time i got a very strange wsdl. ..i would like to call the object (i' ...
Labels :  0
Question :  b'how to correctly make the icon for systemtray in blank using icon sizes of any dimension for systemt' ...
Labels :  0
Question :  b'"is there a way to check a variable that exists in a differen

The labels are 0,1,2 or 3. To see which of these correspond to which string label, you can check class_names property on the dataset.

In [ ]:
for i,label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [ ]:
raw_val_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset ='validation',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [ ]:
test_dir = dataset_dir/'test'
raw_test_ds = preprocessing.text_dataset_from_directory(
    test_dir, batch_size=batch_size)

Found 8000 files belonging to 4 classes.


### Prepare the dataset for training

Next, you will standardize,tokenize,and vectorize the data using the *preprocessing.TextVectorization* layer.



*   Standardization refers to preprocessing the text, typically to remove punctuation or HTML elements to simplify the dataset.

*   Tokennization refers to splitting strings into tokens(for example, splittinig a sentence into individual words by splitting on whitespace).   

*   Vectorization refers to converting tokens into number so they can be fed into a neural network.

All of these tasks can be accomplished with this layer. You can learn more about each of thses in the API doc.



*   The default standardization convert text to lowercase and remove punctuation.

*   The default tokenizer splits on whitespace.

*   The default vectorization mode is int. this outputs integer indices(one per token). This mode can be used to build models that take word order into account. You can also use other modes, like binary, to build bag-of-word models.

Build two modes t learn more about these. First, use the binary model to build a bag-of-words model. Next, you will use the int mode with a 1D ConvNet.








In [ ]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary'
)

For int mode,in addition to maximun vocabulary size, you need to set an explicit maximum sequence length, Which will cause the layer to pad or truncate sequences to exactly sequence_length values.

In [ ]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

Next, you will call adapt to fit the state of the preprocessing layer to the dataset.

In [ ]:
#Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda text, labels:text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

See the result of using these layers to preprocess data:

In [ ]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

In [ ]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [ ]:
# Retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_test_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

Question tf.Tensor(b"how to get 4 unique random numbers in range <0;9>? i want to get 4 unique random floating point numbers in the range &lt;0;9>. how could i do that. is it possible to do this with a single function so i don't need to generate random numbers in a loop?\n", shape=(), dtype=string)
Label tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
print("'binary' vectorized question:",
      binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 1. 1. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [ ]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor(
[[  24    4   41  149  723  247  183    7  427 1928    3   46    4   41
   149  723  247 3676  316  183    7    2  427    1   24  176    3   40
    14    6   11  204    4   40   13   21    5  394   38   50    3  129
    78    4  647  247  183    7    5  123    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0

  Binary mode returns an array denoting which tokens exist at least once in the input, while int mode replace each token by an integer, thus preserving their order. You can lookup the token(string) that each integer corresponds to by calling .get_vocabulary() on the layer.

In [ ]:
print("1289 --->", int_vectorize_layer.get_vocabulary()[1289])
print("313 --->", int_vectorize_layer.get_vocabulary()[313])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

1289 ---> roman
313 ---> source
Vocabulary size: 10000


### Ready to train a model

In [ ]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

### Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

*.cache()* keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a battleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performate on disk cache, Which is more efficient to  read than many small files.

*.prefetch()* overlaps data preprocessing and model execution while training.



In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size = AUTOTUNE)

In [ ]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

### Train the model

It's time to create our neural network. For the binary vectorized data, train a simple bag-of-words linear model:

In [ ]:
binary_model = tf.keras.Sequential([layers.Dense(4)])
binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = 'adam',
    metrics = ['accuracy'])
history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)

Epoch 1/10
200/200 [==============================] - 5s 21ms/step - loss: 1.2433 - accuracy: 0.5219 - val_loss: 0.9177 - val_accuracy: 0.7681
Epoch 2/10
200/200 [==============================] - 1s 4ms/step - loss: 0.8218 - accuracy: 0.8223 - val_loss: 0.7528 - val_accuracy: 0.7950
Epoch 3/10
200/200 [==============================] - 1s 4ms/step - loss: 0.6538 - accuracy: 0.8569 - val_loss: 0.6665 - val_accuracy: 0.8094
Epoch 4/10
200/200 [==============================] - 1s 4ms/step - loss: 0.5538 - accuracy: 0.8834 - val_loss: 0.6126 - val_accuracy: 0.8225
Epoch 5/10
200/200 [==============================] - 1s 4ms/step - loss: 0.4845 - accuracy: 0.9016 - val_loss: 0.5756 - val_accuracy: 0.8319
Epoch 6/10
200/200 [==============================] - 1s 5ms/step - loss: 0.4322 - accuracy: 0.9140 - val_loss: 0.5488 - val_accuracy: 0.8350
Epoch 7/10
200/200 [==============================] - 1s 5ms/step - loss: 0.3907 - accuracy: 0.9267 - val_loss: 0.5285 - val_accuracy: 0.8363
Epoch

Next, you will use the int vectorized layer to build a 1D ConvNet.

In [ ]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding ="valid", activation="relu", strides = 2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [ ]:
# vocab_size is VOCAB_SIZE + 1 since 0 is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer ='adam',
    metrics = ['accuracy'])

history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)


Epoch 1/5
200/200 [==============================] - 8s 38ms/step - loss: 1.3043 - accuracy: 0.3786 - val_loss: 0.7979 - val_accuracy: 0.7038
Epoch 2/5
200/200 [==============================] - 5s 27ms/step - loss: 0.7256 - accuracy: 0.7091 - val_loss: 0.5672 - val_accuracy: 0.7812
Epoch 3/5
200/200 [==============================] - 5s 26ms/step - loss: 0.4595 - accuracy: 0.8425 - val_loss: 0.5062 - val_accuracy: 0.8025
Epoch 4/5
200/200 [==============================] - 5s 27ms/step - loss: 0.2795 - accuracy: 0.9297 - val_loss: 0.4941 - val_accuracy: 0.8169
Epoch 5/5
200/200 [==============================] - 6s 28ms/step - loss: 0.1529 - accuracy: 0.9700 - val_loss: 0.5111 - val_accuracy: 0.8100


Compare the two models:

In [ ]:
print("Liner model on binary vectorized data: ")
print(binary_model.summary())

Liner model on binary vectorized data: 
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
print("ConvNet model on int vectorizer data:")
print(int_model.summary())

ConvNet model on int vectorizer data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


Evaluate both models on the test data:

In [ ]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy : {:2.2%}".format(binary_accuracy))
print("Int model accuracy : {:2.2%}".format(int_accuracy))

250/250 [==============================] - 4s 16ms/step - loss: 0.5386 - accuracy: 0.8048
Binary model accuracy : 81.51%
Int model accuracy : 80.48%


### Export the model

In the code above, applied the TextVectorization layer to yhe dataset before feeding twxt to the model. If you want to make model capable of processing raw strings(for example, to simplify deploying it), you can include the TextVectorization layer inside your model. To do so you can create a new model using the weights you just trained.

In [ ]:
export_model = tf.keras.Sequential(
    [binary_vectorize_layer, binary_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics = ['accuracy'])

# Test it with 'raw_test_ds', which yiels raw string
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(binary_accuracy))


250/250 [==============================] - 4s 13ms/step - loss: 0.5222 - accuracy: 0.8222
Accuracy: 81.51%


Now your model can take raw strings as input and predict a score for each label using *model.predict*. Define a function to find the label with the maximum score

In [ ]:
def get_string_labels(predicted_score_batch):
  predicted_int_labels = tf.argmax(predicted_score_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

### Run interence on new data

In [ ]:
inputs = [
          " public String getDepartment() {return department;", #python
          "debug public static void main(string[] args) {...}", # java
]
predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
  print("Question: ",input)
  print("Predicted label:", label.numpy())

Question:   public String getDepartment() {return department;
Predicted label: b'csharp'
Question:  debug public static void main(string[] args) {...}
Predicted label: b'csharp'


# **Example 2: Predict the author of Illiad translations**

In [ ]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = utils.get_file(name, origin=DIRECTORY_URL + name)

811008/807992 [==============================] - 0s 0us/step


In [ ]:
print(text_dir)

/root/.keras/datasets/butler.txt


In [ ]:
parent_dir = pathlib.Path(text_dir).parent

In [ ]:
print(parent_dir)
print(" ")
list(parent_dir.iterdir())


/root/.keras/datasets
 


[PosixPath('/root/.keras/datasets/derby.txt'),
 PosixPath('/root/.keras/datasets/butler.txt'),
 PosixPath('/root/.keras/datasets/cowper.txt')]

## Load the dataset

You will use `TextLineDataset`, Which is designed to create a `tf.data.Dataset` from a text file in which each example is a line of text from the original file, Whereas `text_dataset_from_directory` treats all contents of a file as a single example.
`TextLineDataset` is useful for text data that is primarily line-based(for example,poetry or error logs).

Iterte through these files, loading each one into its own dataset. Each example needs to be individually labeled, so use `tf.data.Dataset.map` to apply a labeler function to each one. This will iterate over every example in the dataset, returning (example, labels) pairs.

In [ ]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)

In [ ]:
labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(str(parent_dir/file_name))
  labeled_dataset = lines_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

print(labeled_data_sets[0])

<MapDataset shapes: ((), ()), types: (tf.string, tf.int64)>


Next, you'll combine these labeled datasets into a single dataset, and shuffle it.

In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
VALIDATION_SIZE = 5000

In [ ]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

In [ ]:
for text, label in all_labeled_data.take(10):
  print("Sentence: ", text.numpy())
  print("label: ", label.numpy())

Sentence:  b'Within my tent, leaning against the wall,'
label:  0
Sentence:  b'for the mules to pass you. Afterwards when I have taken the body home'
label:  2
Sentence:  b'In beauty far surpassing all their sex:'
label:  1
Sentence:  b'sent on a like design to the camp of the Grecians. From him they are'
label:  1
Sentence:  b'A barrier against all the winds of heaven,'
label:  0
Sentence:  b'With rapid strides he came; the mountains huge'
label:  0
Sentence:  b'The routed sons of Greece should feel how much'
label:  1
Sentence:  b'From Venus, child of Jove; his mother owns'
label:  1
Sentence:  b'From Chiron, justest of the Centaurs, he.'
label:  1
Sentence:  b"Oft as Achilles, swift of foot, essay'd"
label:  1


### Prepare the dataset for training
Instead of using the Keras `TextVectorization` layer to preprocess our text dataset, you will now use the `tf.text API` to standardize and tokenize the data, buid a vocabulary and use `StaticVocabularyTable` to map tokens to integers to feed to the model.

While tf.text provides various tokenizers, you will use the `UnicodeScriptTokenizer`to tokenize our dataset. Defiine a function to convert thr text to lower-case and tokenize it. You will use `tf.data.dataset.map` to apply the tokenization to the dataset.

In [ ]:
tokenizer = tf_text.UnicodeScriptTokenizer()

In [ ]:
def tokenize(text, unused_label):
  lower_case = tf_text.case_fold_utf8(text)
  return tokenizer.tokenize(lower_case)

In [ ]:
tokenized_ds = all_labeled_data.map(tokenize)

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


In [ ]:
for text_batch in tokenized_ds.take(5):
  print("Tokens: ", text_batch.numpy())

Tokens:  [b'within' b'my' b'tent' b',' b'leaning' b'against' b'the' b'wall' b',']
Tokens:  [b'for' b'the' b'mules' b'to' b'pass' b'you' b'.' b'afterwards' b'when'
 b'i' b'have' b'taken' b'the' b'body' b'home']
Tokens:  [b'in' b'beauty' b'far' b'surpassing' b'all' b'their' b'sex' b':']
Tokens:  [b'sent' b'on' b'a' b'like' b'design' b'to' b'the' b'camp' b'of' b'the'
 b'grecians' b'.' b'from' b'him' b'they' b'are']
Tokens:  [b'a' b'barrier' b'against' b'all' b'the' b'winds' b'of' b'heaven' b',']


Next,you will build a vocabulary by sorting tokens by frequency and keeping the top `VOCAB_SIZE` tokens.

In [ ]:
tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  for tok in toks:
    vocab_dict[tok] += 1
print(vocab_dict)

vocab = sorted(vocab_dict.items(), key=lambda x:x[1], reverse = True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)
print("First five vocab entries: ", vocab[:5])



defaultdict(<function <lambda> at 0x7f28f4521d90>, {b'within': 264, b'my': 2230, b'tent': 241, b',': 45478, b'leaning': 15, b'against': 347, b'the': 28299, b'wall': 269, b'for': 3401, b'mules': 71, b'to': 9868, b'pass': 225, b'you': 1901, b'.': 11635, b'afterwards': 10, b'when': 1558, b'i': 3351, b'have': 1262, b'taken': 78, b'body': 340, b'home': 259, b'in': 6568, b'beauty': 42, b'far': 510, b'surpassing': 20, b'all': 3037, b'their': 2382, b'sex': 11, b':': 1712, b'sent': 273, b'on': 3152, b'a': 4327, b'like': 421, b'design': 33, b'camp': 145, b'of': 13489, b'grecians': 150, b'from': 3407, b'him': 3744, b'they': 2621, b'are': 927, b'barrier': 17, b'winds': 66, b'heaven': 319, b'with': 5675, b'rapid': 75, b'strides': 10, b'he': 7004, b'came': 565, b';': 8664, b'mountains': 47, b'huge': 117, b'routed': 21, b'sons': 471, b'greece': 272, b'should': 643, b'feel': 47, b'how': 301, b'much': 267, b'venus': 109, b'child': 152, b'jove': 1376, b'his': 8503, b'mother': 259, b'owns': 5, b'chiron':

To convert the tokens into integers, use the `vocab` set to create a `StaticVocabularyTable`. You will map tokens to integer in the range`[2, vocab_size + 2]`. As with the TextVectorization layer, 0 is resrved to denote padding and 1 is reserved to denote an out-of-vocabulary(OOV) token.



In [ ]:
keys = vocab
values = range(2, len(vocab) + 2) # reserve 0 for padding, 1  for OOV

init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64)

num_oov_buckets = 1
vocab_table =tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

Finally, define a function to standardize, tokenize and vectorize the dataset using the tokenizer and lookup table:

In [ ]:
def preprocess_text(text, label):
  standardized = tf_text.case_fold_utf8(text)
  tokenized = tokenizer.tokenize(standardized)
  vectorized = vocab_table.lookup(tokenized)
  return vectorized, label

In [ ]:
#trying this on single example to see the output:

example_text, example_label = next(iter(all_labeled_data))
print("Sentence :", example_text.numpy())
vectorized_text, example_label = preprocess_text(example_text, example_label)
print("Vectorized sentence: ", vectorized_text.numpy())

Sentence : b'Within my tent, leaning against the wall,'
Vectorized sentence:  [ 263   32  287    2 2500  199    3  257    2]


Now run the preprocess function on the dataset using tf.data.Dataset.map

In [ ]:
all_encoded_data = all_labeled_data.map(preprocess_text)

### Split the dataset into train and test

The Keras `TextVectorization` layer also batches and pads the vectorized data. Padding is required because the example inside of a batch need to be the same size and shape, but the example in these dataset are not all the same size-each line of text has a differrent number of words. `tf.data.Dataset` support splitting and padded-batching dataset:

In [ ]:
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

In [ ]:
train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)

Now, `validation_data` and `train_data` are not collections of (`example, labels`) pairs, but collections of batches. Eacg batch is pair of (many examples, many labels) represented as array.

To illustrate:

In [ ]:
sample_text, sample_labels = next(iter(validation_data))
print("Text batch shape: ", sample_text.shape)
print("Label batch shape: ", sample_labels.shape)
print("First text example: ", sample_text[0])
print("First label example: ", sample_labels[0])


Text batch shape:  (64, 18)
Label batch shape:  (64,)
First text example:  tf.Tensor(
[ 263   32  287    2 2500  199    3  257    2    0    0    0    0    0
    0    0    0    0], shape=(18,), dtype=int64)
First label example:  tf.Tensor(0, shape=(), dtype=int64)


Since we use `0` for padding and `1` for out-of-vacabulary(OOV)tokens,the vocabulary size has increased by two.

In [ ]:
vocab_size += 2

Configure the datasets for better performance as before.

In [ ]:
trian_data = configure_dataset(train_data)
validation_data = configure_dataset(validation_data)

### Train the model

You can train a model on this dataset as before.

In [ ]:
model = create_model(vocab_size = vocab_size, num_labels=3)
model.compile(
    optimizer = 'adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy'])

history = model.fit(train_data, validation_data=validation_data, epochs=3)

Epoch 1/3
697/697 [==============================] - 35s 19ms/step - loss: 0.6972 - accuracy: 0.6635 - val_loss: 0.3805 - val_accuracy: 0.8320
Epoch 2/3
697/697 [==============================] - 32s 13ms/step - loss: 0.2827 - accuracy: 0.8909 - val_loss: 0.3564 - val_accuracy: 0.8490
Epoch 3/3
697/697 [==============================] - 32s 13ms/step - loss: 0.1888 - accuracy: 0.9320 - val_loss: 0.3842 - val_accuracy: 0.8428


In [ ]:
loss, accuracy = model.evaluate(validation_data)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

79/79 [==============================] - 1s 2ms/step - loss: 0.3842 - accuracy: 0.8428
Loss:  0.38415566086769104
Accuracy: 84.28%


### Export the model

To make our model capable to taking raw strings as imput, you will create a `TextVectorization`layer that performs the same steps as our custom preprocessing function. Since you already trained a vocabulary, you can use `set_vocabulary` instead of `adapt` which trains a new vocabulary.

In [ ]:
preprocess_layer = TextVectorization(
    max_tokens=vocab_size,
    standardize=tf_text.case_fold_utf8,
    split=tokenizer.tokenize,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)
preprocess_layer.set_vocabulary(vocab)

In [ ]:
export_model = tf.keras.Sequential(
    [preprocess_layer, model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
# Create a test dataset of raw string

test_ds = all_labeled_data.take(VALIDATION_SIZE).batch(BATCH_SIZE)
test_ds = configure_dataset(test_ds)
loss, accuracy = export_model.evaluate(test_ds)
print("Loss: ",loss)
print("Accuracy: {:2.2%}".format(accuracy))

79/79 [==============================] - 8s 11ms/step - loss: 0.5079 - accuracy: 0.7979
Loss:  0.5135167241096497
Accuracy: 79.38%


The loss and accuracy for the model on encoded validation set and the exported model on the raw validation set are the same, as expeted.


## Run inference on the data

In [ ]:
inputs=[
       "Join'd to th' Ionians with their flowing robes,",  # Label: 1
       "the allies, and his armour flashed about him so that he seemed to all",  # Label: 2
       "And with loud clangor of his arms he fell.",  # Label: 0
]

predicted_scores = export_model.predict(inputs)
predicted_labels = tf.argmax(predicted_scores, axis=1)
for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted labels: ", label.numpy())

Question:  Join'd to th' Ionians with their flowing robes,
Predicted labels:  1
Question:  the allies, and his armour flashed about him so that he seemed to all
Predicted labels:  2
Question:  And with loud clangor of his arms he fell.
Predicted labels:  0


### *Downloading more datasets using TensorFlow Datasets(TFDS)*

https://www.kaggle.com/maximgolovatchev/imdb-reviews/notebook


In [ ]:
train_ds = tfds.load(
    'imdb_reviews',
    split='train',
    batch_size = BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZVO3E7/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZVO3E7/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZVO3E7/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
val_ds = tfds.load(
    'imdb_reviews',
    split = 'train',
    batch_size=BATCH_SIZE,
    shuffle_files = True,
    as_supervised=True)

Print a few examples.

In [ ]:
for review_batch, label_batch in val_ds.take(1):
  for i in range(5):
    print("Review: ", review_batch[i].numpy())
    print("Label: ",label_batch[i].numpy())


Review:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Label:  0
Review:  b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbi

Prepare the dataset for training

In [ ]:
vectorize_layer = TextVectorization(
    max_tokens = VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda text, labels: text)
vectorize_layer.adapt(train_text)

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [ ]:
train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)

In [ ]:
# Configure dataset for performance as before

train_ds = configure_dataset(train_ds)
val_ds = configure_dataset(val_ds)

### Train the model

In [ ]:
model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=1)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          640064    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 660,673
Trainable params: 660,673
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
    loss=losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

Epoch 1/3
391/391 [==============================] - 25s 61ms/step - loss: 0.6124 - accuracy: 0.5799 - val_loss: 0.3031 - val_accuracy: 0.8816
Epoch 2/3
391/391 [==============================] - 19s 50ms/step - loss: 0.3150 - accuracy: 0.8592 - val_loss: 0.1744 - val_accuracy: 0.9449
Epoch 3/3
391/391 [==============================] - 20s 50ms/step - loss: 0.1962 - accuracy: 0.9228 - val_loss: 0.0948 - val_accuracy: 0.9782


In [ ]:
loss, accuracy = model.evaluate(val_ds)

print("loss: ",loss)
print("Accuracy: {:2.2%}".format(accuracy))

391/391 [==============================] - 4s 10ms/step - loss: 0.0948 - accuracy: 0.9782
loss:  0.09483323246240616
Accuracy: 97.82%


### Export the model

In [ ]:
export_model = tf.keras.Sequential(
    [vectorize_layer, model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
inputs = [
    "This is a fantastic movie.",
    "This is a bad movie.",
    "This movie was so bad that it was good.",
    "I will never say yes to watching this movie.",
] 

predicted_scores = export_model.predict(inputs)
predicted_labels = [int(round(x[0])) for x in predicted_scores]
for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("predicted labels: ",label)

Question:  This is a fantastic movie.
predicted labels:  1
Question:  This is a bad movie.
predicted labels:  0
Question:  This movie was so bad that it was good.
predicted labels:  0
Question:  I will never say yes to watching this movie.
predicted labels:  1


In [ ]:
print(predicted_scores)
print(predicted_labels)

[[0.99992895]
 [0.1504317 ]
 [0.36695465]
 [0.67529476]]
[1, 0, 0, 1]
